In [39]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
from pyspark.sql.window import Window
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2
import matplotlib.pyplot as plt
import pandas as pd
spark = SparkSession.builder \
    .appName("Analysis_23") \
    .getOrCreate()

In [49]:
df= (
    spark.read.csv(
        "../dataset/New/nbs.csv", 
        header=True,
        inferSchema=True
    )
    .withColumnRenamed("Fac","FAC NO")
)

validation= (
    spark.read.csv(
        "../dataset/New/boss.csv", 
        header=True,
        inferSchema=True
    )
#     .filter(~f.col("CRIB_assign").isin[-400])
    .filter(~f.col("CRIB_Assigned").isin([-400]))
    .withColumn("total_score_cat",
                f.when(f.col("Total_Score")<0,"below 0")
                 .when(f.col("Total_Score").between(0,40),"0-40")
#                 .when(f.col("Total_Score").between(10,20),"10-20")
#                 .when(f.col("Total_Score").between(20,30),"20-30")
#                 .when(f.col("Total_Score").between(30,40),"30-40")
                .when(f.col("Total_Score").between(40,50),"40-50")
                .when(f.col("Total_Score").between(50,60),"50-60")
                .when(f.col("Total_Score").between(60,70),"60-70")
                .when(f.col("Total_Score").between(70,80),"70-80")
                .when(f.col("Total_Score").between(80,90),"80-90")
                .when(f.col("Total_Score")>=90,"above90")
               )
    .join(df,"FAC NO","left")
)

validation6=(
   validation
#     .filter(~f.col("CRIB_assign").isin[-400])
    .filter(~f.col("Credit score").isin([-400]))
    .withColumn("Credit_score_cat",
                f.when(f.col("Credit score")<0,"below 0")
                 .when(f.col("Credit score").between(0,40),"0-40")
#                 .when(f.col("Total_Score").between(10,20),"10-20")
#                 .when(f.col("Total_Score").between(20,30),"20-30")
#                 .when(f.col("Total_Score").between(30,40),"30-40")
                .when(f.col("Credit score").between(40,50),"40-50")
                .when(f.col("Credit score").between(50,60),"50-60")
                .when(f.col("Credit score").between(60,70),"60-70")
                .when(f.col("Credit score").between(70,80),"70-80")
                .when(f.col("Credit score").between(80,90),"80-90")
                .when(f.col("Credit score")>=90,"above90")
               )
   
)


In [50]:
validation6.show()

+-------------------+---+------+-------+--------------------+-------------------+------+---------+-----------+---+----------------+----+---+-------------+----------+---------+---------+----------------+----------------+----------+---------------------+---------------------+---------------+---------------+----------------+------------------+-------------------+---------------+------------+----------------+
|             FAC NO|_c0|BRN CD|PROCODE|        CUST_PERMRES|       GUAR_PERMRES|EXP_CD|   DSCR_C|connex_date|DPD|90>=day_contract|good|bad|      Product|Conex Date|Portfolio|TOTAL DUE|TOTAL COLLECTION|Collection Ratio|CRIB_SCORE|CUST_PERMRES_Assigned|GUAR_PERMRES_Assigned|EXP_CD_Assigned|DSCR_C_Assigned|PROCODE_Assigned|     CRIB_Assigned|        Total_Score|total_score_cat|Credit score|Credit_score_cat|
+-------------------+---+------+-------+--------------------+-------------------+------+---------+-----------+---+----------------+----+---+-------------+----------+---------+-------

In [52]:
validation_group=(validation6
                  .groupBy("total_score_cat")
                  .agg(
                      f.count("FAC NO").alias("Total_facility_count"),
                      f.sum("90>=day_contract").alias("NPA"),
                      f.sum("Portfolio").alias("Portfolio"),
                      f.sum("TOTAL COLLECTION").alias("TOTAL COLLECTION"),
                        f.sum("TOTAL DUE").alias("TOTAL DUE")
                  )
                  .withColumn("NPA_rate",f.col("NPA")/f.col("Total_facility_count"))

                  
                 )
validation_group2=(validation6
                   .groupBy("total_score_cat","90>=day_contract")
                   .agg(
                       f.sum("Portfolio").alias("NPL_Portfolio"))
                   .filter(f.col("90>=day_contract")==1)
                   .select("total_score_cat","NPL_Portfolio")
                   )
validation_group3=(
    validation_group.join(validation_group2,"total_score_cat","left")
    .withColumn("Total_collection_ratio",f.col("TOTAL COLLECTION")/f.col("TOTAL DUE"))
    .withColumn("NPL_ratio",f.col("NPL_Portfolio")/f.col("Portfolio"))
)           

In [43]:
validation_group3.show()

+---------------+--------------------+---+----------+----------------+---------+-------------------+-------------+----------------------+-------------------+
|total_score_cat|Total_facility_count|NPA| Portfolio|TOTAL COLLECTION|TOTAL DUE|           NPA_rate|NPL_Portfolio|Total_collection_ratio|          NPL_ratio|
+---------------+--------------------+---+----------+----------------+---------+-------------------+-------------+----------------------+-------------------+
|          40-50|                   1|  0|    362136|          202500|   219375|                0.0|         NULL|    0.9230769230769231|               NULL|
|          80-90|                 626| 61| 196350741|       139574808|154682692|0.09744408945686901|     25461209|    0.9023298353250796| 0.1296720800254072|
|          60-70|                 301| 49| 101938858|        68693317| 78753364|0.16279069767441862|     19519928|    0.8722588282070084| 0.1914866262284398|
|           0-40|                  11|  1|   4972987

In [53]:
validation_group3.toPandas().to_csv("../output/20240126_final_output/validation_new model.csv")

In [55]:
validation_group=(validation6
                  .groupBy("Credit_score_cat")
                  .agg(
                      f.count("FAC NO").alias("Total_facility_count"),
                      f.sum("90>=day_contract").alias("NPA"),
                      f.sum("Portfolio").alias("Portfolio"),
                      f.sum("TOTAL COLLECTION").alias("TOTAL COLLECTION"),
                        f.sum("TOTAL DUE").alias("TOTAL DUE")
                  )
                  .withColumn("NPA_rate",f.col("NPA")/f.col("Total_facility_count"))

                  
                 )
validation_group2=(validation6
                   .groupBy("Credit_score_cat","90>=day_contract")
                   .agg(
                       f.sum("Portfolio").alias("NPL_Portfolio"))
                   .filter(f.col("90>=day_contract")==1)
                   .select("Credit_score_cat","NPL_Portfolio")
                   )
validation_group3=(
    validation_group.join(validation_group2,"Credit_score_cat","left")
    .withColumn("Total_collection_ratio",f.col("TOTAL COLLECTION")/f.col("TOTAL DUE"))
    .withColumn("NPL_ratio",f.col("NPL_Portfolio")/f.col("Portfolio"))
)           

In [56]:
validation_group3.toPandas().to_csv("../output/20240126_final_output/validation_old_model.csv")